In [3]:
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad")

# Check the structure of the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [4]:
# Display a sample question-answer pair
sample = dataset['train'][0]
print(f"Question: {sample['question']}")
print(f"Context: {sample['context']}")
print(f"Answer: {sample['answers']['text'][0]}")

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Answer: Saint Bernadette Soubirous


In [5]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Preprocessing function to tokenize questions and contexts
def preprocess_function(examples):
    return tokenizer(
        examples['question'], 
        examples['context'], 
        truncation=True, 
        padding="max_length", 
        max_length=384
    )

# Apply the preprocessing function to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [6]:
train_dataset = tokenized_dataset["train"]
validation_dataset = tokenized_dataset["validation"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(validation_dataset)}")

Train dataset size: 87599
Validation dataset size: 10570


In [7]:
def preprocess_function(examples):
    inputs = tokenizer(
        examples['question'], 
        examples['context'], 
        truncation=True, 
        padding="max_length", 
        max_length=384
    )
    
    start_positions = []
    end_positions = []
    
    for i, context in enumerate(examples['context']):
        # Get the first answer and its start position
        answer = examples['answers'][i]['text'][0]  # Use the first answer only
        answer_start = examples['answers'][i]['answer_start'][0]  # Use the first start position

        # Compute end position
        answer_end = answer_start + len(answer)

        # Convert character positions to token positions
        start_position = inputs.char_to_token(i, answer_start)
        end_position = inputs.char_to_token(i, answer_end - 1)

        # Handle cases where the tokenization truncates the answer
        if start_position is None:
            start_position = tokenizer.model_max_length
        if end_position is None:
            end_position = tokenizer.model_max_length

        start_positions.append(start_position)
        end_positions.append(end_position)
    
    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions
    return inputs

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [9]:
import torch
from transformers import AutoModelForQuestionAnswering, Trainer, TrainingArguments

# Check if MPS (Apple Silicon) is available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Load DistilBERT model and move to MPS
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased").to(device)

# Select a smaller portion of the training and validation datasets
small_train_dataset = tokenized_dataset["train"].select(range(30000))
small_validation_dataset = tokenized_dataset["validation"]

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    max_steps=500,  # Train for 50 steps
    report_to="none",  # Disable logging/reporting to avoid slowing down
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_validation_dataset,
)

# Start training
trainer.train()

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
# Select a few examples from the validation set to test
print("Selecting examples from the validation set...")
n_examples = 5
validation_examples = tokenized_dataset["validation"].select(range(n_examples))

In [13]:
import torch

# Check if MPS (Apple Silicon) is available and move model to MPS if available
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)  # Move the model to MPS if available

# Run predictions on the selected validation examples
for i, example in enumerate(validation_examples):
    # Tokenize the input and move it to the same device
    inputs = tokenizer(
        example['question'], example['context'],
        return_tensors="pt", truncation=True, padding="max_length", max_length=384
    ).to(device)  # Move input tensors to MPS device if available

    # Model inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the predicted start and end positions
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)
    middle_index = (start_index + end_index) // 2

    # Convert token predictions back to original text
    input_ids = inputs["input_ids"].squeeze().cpu()  # Move back to CPU for decoding
    

    # input_ids = inputs["input_ids"].squeeze().cpu()  # Move back to CPU for decoding
    # predicted_answer = tokenizer.decode(input_ids[start_index:end_index + 1], skip_special_tokens=True)
    
    # Print the question, predicted answer, and actual answer
    print(f"Example {i + 1}")
    print(f"Question: {example['question']}")
    print(f"Context: {example['context']}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Actual Answer: {example['answers']['text'][0]}")
    print("-" * 50)
    p

Example 1
Question: Which NFL team represented the AFC at Super Bowl 50?
Context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Predicted Answer: which nfl team represented the afc
Actual Answer: Denver Broncos
--------------------------------------------------
Example 